# Data Wrangling - Stock market's Data

**Author**:  Sathish Manthani  (smanthani@my365.bellevue.edu)  
**Date**  :  02/25/2020  
**Course**:  DSC540 - Data Preparation  

## Import required libraries

In [1]:
# Importing requests, json modules which helps to connect to REST APIs and parse the data
import requests, json
# Importing datetime method for date conversion
from datetime import datetime
import logging #for logging connectivity
import configparser #configure the logger
import pandas as pd #for dataframes

# Logging configuration
logging.basicConfig(filename='./Stocks_data_pull_%s.log' %
                        datetime.strftime(datetime.now(), '%m%d%Y_%H%M%S'),
                        level=logging.DEBUG,
                        format='%(asctime)s %(message)s',
                        datefmt='%m-%d %H:%M:%S')

## Method to get list of stock tickers from API

In [2]:
#Import contextlib and csv modules to parse csv data
from contextlib import closing
import csv

def getListofTickers(tickers_url):
    '''
    This method takes REST API url and gets list of tickers. 
    API returns CSV file output and this method returns the list of tickers.
    '''
    logging.debug("[INFO]: Getting Tickers' data via API...")
    with closing(requests.get(tickers_url, stream=True)) as r:
        f = (line.decode('utf-8') for line in r.iter_lines())
        try:
            reader = csv.reader(f, delimiter=',', quotechar='"')
            next(reader, None)    
            tickers_list = []
            for row in reader:
                tickers_list.append(row[0])
        except requests.exceptions.HTTPError as exp:
            logging.debug('[ERROR]: Error connecting to the API.',str(exp))
            return str(exp)
    logging.debug('[INFO]: List of tickers fetched.')
    return tickers_list

## Method to get Stocks' data from API

In [3]:
def getStocksList(url,tickers):
    '''
    This method takes REST API url and tickers list as input and establishes connection. 
    Fetches response data and processes JSON data into Python dictionary and return the response.
    '''
    tickers_list = ','.join(tickers)
    logging.debug("[INFO]: Connecting to the Stocks' data API...")
    # Connecting to the API
    response = requests.get(url+tickers_list)
    logging.debug('[INFO]: Connected to the API successfully! Fetching data...')
    # Raise exception in case of connection error and return appropriate error message.
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as exp:
        logging.debug('[ERROR]: Error connecting to the API.',str(exp))
        return str(exp) 
    
    # If the connection succeeds then read the fetched JSON string
    json_resp = response.json()
    #json_out = json.dumps(json_resp, indent=4)
    logging.debug('[INFO]: Stocks data fetch is complete.')
    return json_resp

## API variables

**I used two APIs for this exercise.  
One fetches the list of tickers.   
Another one gets the stocks data for these tickers.**

In [4]:
# API to get list of tickers
tickers_url = "https://pkgstore.datahub.io/core/nasdaq-listings/nasdaq-listed-symbols_csv/data/595a1f263719c09a8a0b4a64f17112c6/nasdaq-listed-symbols_csv.csv"

In [5]:
# API to get Stocks data for given tickers
stocks_url = "https://financialmodelingprep.com/api/v3/quote/"

## API calls and parsing the output

In [6]:
# Call the method to get list of tickers
tickers_list = getListofTickers(tickers_url)

In [7]:
# Call the method to get stocks' data
# I'm only retrieving the data for a 1500 tickers. 
# Pls note its not necessary the data would be available for all the tickers.
json_resp = getStocksList(stocks_url,tickers_list[:1500])

In [8]:
# Normalize the json output and store it as dataframe
from pandas.io.json import json_normalize
df = json_normalize(json_resp)
#json_normalize shuffles the columns. So,I ordered the columns as given in the API
df = df.reindex(columns=list(json_resp[0].keys()))

## Data preparation

### List of columns in the dataset

In [9]:
#list of columns
df.columns

Index(['symbol', 'price', 'changesPercentage', 'change', 'dayLow', 'dayHigh',
       'yearHigh', 'yearLow', 'marketCap', 'priceAvg50', 'priceAvg200',
       'volume', 'avgVolume', 'exhange', 'open', 'previousClose', 'eps', 'pe',
       'earningsAnnouncement', 'sharesOutstanding', 'timestamp'],
      dtype='object')

### Rename the columns

In [10]:
# Rename the columns to more meaningful names
df = df.rename(columns = {"symbol":"Stock_Ticker", 
                     "price":"Price",
                     "changesPercentage":"Change_%",
                    "change":"Change_$",
                    "dayLow":"24hrs_Low",
                    "dayHigh":"24hrs_High",
                    "yearHigh":"52weeks_High",
                    "yearLow":"52weeks_Low",
                    "marketCap":"Market_Cap",
                    "priceAvg50":"Avg_50days_Price",
                    "priceAvg200":"Avg_200days_Price",
                     "volume":"Volume",
                     "avgVolume":"Avg_Volume",
                     "exhange":"Exchange",
                     "open":"Open",
                     "previousClose":"Previous_Close", 
                    "eps":"EPS", 
                    "pe":"PE",
                    "earningsAnnouncement":"Earnings_Date", 
                    "sharesOutstanding":"Outstanding_Shares", 
                    "timestamp":"Timestamp"})

### Check the datatypes of the columns

In [11]:
#data types
df.dtypes

Stock_Ticker           object
Price                 float64
Change_%              float64
Change_$              float64
24hrs_Low             float64
24hrs_High            float64
52weeks_High          float64
52weeks_Low           float64
Market_Cap            float64
Avg_50days_Price      float64
Avg_200days_Price     float64
Volume                float64
Avg_Volume            float64
Exchange               object
Open                  float64
Previous_Close        float64
EPS                   float64
PE                    float64
Earnings_Date          object
Outstanding_Shares    float64
Timestamp               int64
dtype: object

### Drop Null values

In [12]:
#Remove rows with null values
df = df.dropna(how='any') 
df.dropna(how='any', inplace=True)

### Convert Object to Date

In [13]:
def castAsDate(date_obj):
    '''
    Cast date object to Date datatype
    '''
    dt = pd.to_datetime(date_obj,infer_datetime_format=True).dt.date
    return dt

### Convert Date object to date

In [14]:
#Earnings date prior to conversion
df["Earnings_Date"].head()

0    2020-01-23T12:30:11.000+0000
3    2020-02-27T21:15:00.000+0000
4    2020-01-28T21:30:00.000+0000
7    2020-01-23T22:13:20.000+0000
8    2019-11-06T02:00:00.000+0000
Name: Earnings_Date, dtype: object

In [15]:
#Conversion
df["Earnings_Date"] = castAsDate(df["Earnings_Date"])
df["Earnings_Date"].head()

0    2020-01-23
3    2020-02-27
4    2020-01-28
7    2020-01-23
8    2019-11-06
Name: Earnings_Date, dtype: object

### Convert Market Cap to Millions 

In [16]:
df["Market_Cap"] = (df.Market_Cap/1000000).map(lambda x: '{:.2f}'.format(x))

### Convert Outstanding shares to 1000s

In [17]:
df["Outstanding_Shares"] = (df.Outstanding_Shares/1000).map(lambda x: '{:.0f}'.format(x))

## Display the data

In [18]:
#Display final output
df.head(50)

,Stock_Ticker,Price,Change_%,Change_$,24hrs_Low,24hrs_High,52weeks_High,52weeks_Low,Market_Cap,Avg_50days_Price,...,Volume,Avg_Volume,Exchange,Open,Previous_Close,EPS,PE,Earnings_Date,Outstanding_Shares,Timestamp
0,AAL,22.31,-3.50,-0.810,22.1100,23.7500,35.96,22.11,9505.38,27.691177,...,18094766.0,9193475.0,NASDAQ,23.5700,23.120,3.790,5.886543,2020-01-23,428200,1582783742
3,AAON,53.12,0.19,0.100,53.0800,54.4200,57.55,39.50,2767.42,53.732647,...,223930.0,134975.0,NASDAQ,53.2600,53.020,0.960,55.333332,2020-02-27,52098,1582783742
4,AAPL,292.65,1.59,4.570,286.5000,297.8800,327.85,169.50,1280484.18,315.339100,...,48255442.0,31587619.0,NASDAQ,286.5300,288.080,12.595,23.235410,2020-01-28,4375480,1582783742
7,ABCB,36.70,-3.42,-1.300,36.6400,38.2500,44.90,32.91,2550.79,41.316765,...,630719.0,319680.0,NASDAQ,38.2500,38.000,2.750,13.345455,2020-01-23,69504,1582783742
8,ABDC,8.90,-3.26,-0.300,8.7973,9.2300,9.50,6.98,114.59,9.202392,...,241414.0,31421.0,NASDAQ,9.2300,9.200,0.538,16.542751,2019-11-06,12876,1582783742
10,ABMD,154.09,-2.97,-4.710,154.0200,161.2700,352.87,154.01,6943.71,178.067640,...,743778.0,859126.0,NASDAQ,159.5800,158.800,5.331,28.904520,2020-02-06,45155,1582783742
12,ACGL,43.83,-1.26,-0.560,43.7700,45.0300,48.32,31.53,17778.28,45.395294,...,1338893.0,1290722.0,NASDAQ,44.6200,44.390,3.870,11.325582,2019-07-30,405619,1582783742
15,ACIW,29.80,-3.37,-1.040,29.7200,31.3791,39.37,28.02,3444.85,35.507057,...,524624.0,572109.0,NASDAQ,31.0000,30.840,0.833,35.774307,2020-02-27,115599,1582783742
16,ACLS,23.83,2.45,0.570,23.3200,24.0400,29.24,13.99,776.50,25.666470,...,219450.0,243116.0,NASDAQ,23.3800,23.260,0.500,47.660000,2020-02-05,32585,1582783742
17,ACNB,33.81,-0.06,-0.020,33.1900,34.4883,40.98,31.83,293.29,34.893826,...,32623.0,16193.0,NASDAQ,33.5400,33.830,3.360,10.062501,2020-01-28,8675,1582783742
